# Practice with ipywidgets, Pandas, Matplotlib and Jupyter Notebook, yay!


## Import Libraries

In [1]:
import ipywidgets as widgets
from ipywidgets import interact_manual
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Read CSV and Add Datetime Column

In [2]:
test = pd.read_csv('test_data.txt')
test['Date'] = pd.to_datetime(test[['Year', 'Month']].assign(DAY=1))
#test.head()

## Create Widget to Select Desired Companies

In [3]:
select_companies = widgets.SelectMultiple(
    options= sorted(set(test['Company Name'])),
    value=['COMP-1'],
    rows=len(sorted(set(test['Company Name']))),
    description='Companies:',
    disabled=False
)

## Create Widget to Select Desired Timeframe

In [4]:
dates = sorted(set(test['Date']))
options = [(i.strftime('%b''%Y'), i) for i in dates]
time_frame = widgets.SelectionRangeSlider(
    options=options,
    index=(0,len(options)-1),
    description='Date Range:',
    disabled=False,
    layout={'width': '400px'}
)

## Create Widget to Select Desired Plot Type

In [5]:
dropdown = widgets.Dropdown(
    options= ['Line Plot', 'Stacked Bar', 'Donut', 'Pie Chart'],
    value='Donut',
    description='Chart Types:',
    disabled=False,
)

## Include Output Widget

In [6]:
out = widgets.Output()

## Make Function to House Different Plots

In [9]:
def make_plot():
    out.clear_output
    if dropdown.value == 'Pie Chart':
        with out:
            labels = select_companies.value
            runs = []
            for comp in labels:
                comp_runs = test.loc[(test['Company Name'] == comp) 
                                     & (test['Date'] >= time_frame.value[0]) 
                                     & (test['Date'] <=time_frame.value[1])]
                runs.append(sum(comp_runs['Num Runs']))
            fig1, axl = plt.subplots(figsize=(12, 6))
            axl.pie(runs, autopct=lambda p : '{:.2f}% \n ({:,.0f})'.format(p,p * sum(runs)/100), 
                    shadow=True, startangle=90, textprops=dict(color='w', fontweight='bold'),
                   pctdistance=.70)
            axl.axis('equal')
            axl.legend(labels,
                      title="Companies",
                      loc="center left",
                      bbox_to_anchor=(1, 0, 0.5, 1))
            plt.title('Runs by Company between {}-{}'.format((time_frame.value[0]).strftime('%b''%Y'),
                                                                (time_frame.value[1]).strftime('%b''%Y')))
            plt.tight_layout()
    elif dropdown.value == 'Donut':
        with out:
            fig, ax = plt.subplots(figsize=(12, 6), subplot_kw=dict(aspect="equal"))

            labels = list(select_companies.value)
            runs = []
            for comp in labels:
                comp_runs = test.loc[(test['Company Name'] == comp) 
                                     & (test['Date'] >= time_frame.value[0]) 
                                     & (test['Date'] <=time_frame.value[1])]
                runs.append(sum(comp_runs['Num Runs']))
            labels = [f'{l} - {s} Runs ({s*100/sum(runs):.2f}%)' for s,l in zip(runs, labels)]
            wedges, texts = ax.pie(runs, wedgeprops=dict(width=0.5), startangle=-30)

            bbox_props = dict(boxstyle="square,pad=0.3", fc="w", ec="k", lw=0.72)
            kw = dict(arrowprops=dict(arrowstyle="-"),
                      bbox=bbox_props, zorder=0, va="center")
            for i, p in enumerate(wedges):
                ang = (p.theta2 - p.theta1)/2. + p.theta1
                y = np.sin(np.deg2rad(ang))
                x = np.cos(np.deg2rad(ang))
                horizontalalignment = {-1: "right", 1: "left"}[int(np.sign(x))]
                connectionstyle = "angle,angleA=0,angleB={}".format(ang)
                kw["arrowprops"].update({"connectionstyle": connectionstyle})
                ax.annotate(labels[i], xy=(x, y), xytext=(1.2*np.sign(x), 1.4*y),
                            horizontalalignment=horizontalalignment, **kw)
            ax.set_title("Runs by Company between {}-{}".format((time_frame.value[0]).strftime('%b''%Y'),
                                                                (time_frame.value[1]).strftime('%b''%Y')))
    elif dropdown.value == 'Line Plot':
        with out:
            temp_test =  test.loc[(test['Company Name'].isin(select_companies.value))]
            temp_test = temp_test.pivot(index='Date', columns='Company Name', values='Num Runs')
            temp_test[time_frame.value[0]:time_frame.value[1]].plot(figsize=(12, 6), 
                                                                    title= 'Runs Per Company By Month', 
                                                                    ylabel='Num Runs').legend(bbox_to_anchor=(1.05,1), 
                                                                                              loc='upper left')
    elif dropdown.value == 'Stacked Bar':
        with out:
            temp_test =  test.loc[(test['Company Name'].isin(select_companies.value))]
            temp_test = temp_test.pivot(index='Date', columns='Company Name', values='Num Runs')
            temp_test[time_frame.value[0]:time_frame.value[1]].plot(kind= 'bar', stacked= True, figsize=(12, 6), 
                                                                    title= 'Runs Per Month By Company', 
                                                                    ylabel='Num Runs').legend(bbox_to_anchor=(1.05,1), 
                                                                                              loc='upper left')

## Put It All Together

In [10]:
display(select_companies)
display(time_frame)
display(dropdown)
interact_manual(make_plot)
display(out)

SelectMultiple(description='Companies:', index=(0, 1, 2, 3, 4, 5, 6, 7), options=('COMP-1', 'COMP-2', 'COMP-3'…

SelectionRangeSlider(description='Date Range:', index=(0, 9), layout=Layout(width='400px'), options=(('Apr2020…

Dropdown(description='Chart Types:', index=3, options=('Line Plot', 'Stacked Bar', 'Donut', 'Pie Chart'), valu…

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

Output()

Things to improve:
1. (solved!) Figure out a way to have Chart Type update without re-running the cell 
2. Give companies their own unique color so they don't change when the desired companies selected change
3. (solved!) Add date range selected to Pie and Donut plots
5. Figure out why date on Stacked Bar plot looks so bad when it's based on line plot code....
        https://stackoverflow.com/questions/30133280/pandas-bar-plot-changes-date-format
6. Make stacked bar plot more understandable with multiple companies selected?
7. Pie and donut still have overlapping labels sometimes...
8. Use two DatePicker widgets to select start and end dates for data with longer timeframe?
9. Prettier plots...
